In [30]:
import numpy as np
import pandas as pd
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score, classification_report
import keras

In [31]:
data = pd.read_csv(r'C:\Users\Jubin\Desktop\RP18\Datasets\intel\interpolated\injected_malfunction\mote=1_sensortype=temperature_faulttype=malfunction.csv')

In [32]:
X = data.drop(columns=['timestamp', 'mote_id', 'light', 'has_fault_type'])
Y = data['has_fault_type']

In [33]:
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

In [34]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [35]:
# Split the dataset into training, validation, and test sets
X_train, X_temp, Y_train, Y_temp = train_test_split(X_scaled, Y_encoded, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

In [36]:
# Wavelet denoising
def wavelet_denoise(data):
    denoised_data = []
    for row in data:
        coeffs = pywt.wavedec(row, 'db4', level=3)  # Adjust the wavelet and level as needed
        thresholded_coeffs = [pywt.threshold(detail, 0.5, mode='soft') for detail in coeffs[1:]]
        denoised_signal = pywt.waverec([coeffs[0]] + thresholded_coeffs, 'db4')  # Adjust the wavelet as needed
        denoised_data.append(denoised_signal)
    return np.array(denoised_data)

In [37]:
X_train_denoised = wavelet_denoise(X_train)
X_val_denoised = wavelet_denoise(X_val)
X_test_denoised = wavelet_denoise(X_test)

c:\Users\Jubin\AppData\Local\Programs\Python\Python37\lib\site-packages\pywt\_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))


In [38]:
# Build a neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_denoised.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # For binary classification, adjust for multi-class

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

c:\Users\Jubin\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [39]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, save_weights_only=False, monitor='val_loss', mode='min', verbose=1)

In [40]:
# Train the model
history = model.fit(X_train_denoised, Y_train, epochs=100, batch_size=32, validation_data=(X_val_denoised, Y_val), callbacks=[early_stopping, model_checkpoint])

Epoch 1/100


414/441 [===========================>..] - ETA: 0s - loss: 0.4869 - accuracy: 0.8172
Epoch 1: val_loss improved from inf to 0.46379, saving model to best_model.h5
441/441 [==============================] - 1s 1ms/step - loss: 0.4856 - accuracy: 0.8171 - val_loss: 0.4638 - val_accuracy: 0.8181
Epoch 2/100
400/441 [==========================>...] - ETA: 0s - loss: 0.4641 - accuracy: 0.8212
Epoch 2: val_loss improved from 0.46379 to 0.45388, saving model to best_model.h5
441/441 [==============================] - 0s 1ms/step - loss: 0.4655 - accuracy: 0.8202 - val_loss: 0.4539 - val_accuracy: 0.8228
Epoch 3/100
407/441 [==========================>...] - ETA: 0s - loss: 0.4579 - accuracy: 0.8213
Epoch 3: val_loss did not improve from 0.45388
441/441 [==============================] - 0s 1ms/step - loss: 0.4543 - accuracy: 0.8236 - val_loss: 0.4548 - val_accuracy: 0.8228
Epoch 4/100
389/441 [=========================>....] - ETA: 0s - loss: 0.4471 - accuracy: 0.8266
Epoch 4: val_loss improv

In [41]:
# Load the best model
best_model = keras.models.load_model('best_model.h5')

In [42]:
# Evaluate the model
Y_pred = best_model.predict(X_test_denoised)
Y_pred = (Y_pred > 0.5).astype(int)  # Adjust the threshold for binary classification

95/95 [==============================] - 0s 683us/step


In [43]:
accuracy = accuracy_score(Y_test, Y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Print classification report
print(classification_report(Y_test, Y_pred))

Accuracy: 0.84
              precision    recall  f1-score   support

           0       0.84      1.00      0.91      2499
           1       1.00      0.10      0.17       525

    accuracy                           0.84      3024
   macro avg       0.92      0.55      0.54      3024
weighted avg       0.87      0.84      0.78      3024

